In [91]:
from functions import *
import glob
from PIL import Image
import os
from sklearn.model_selection import train_test_split

In [87]:
IMAGE_DIR = "./data/data/data"

def load():
    file_list = glob.glob(IMAGE_DIR + "/*.jpg")
    X = []
    Y = []

    for fname in file_list:
        with Image.open(fname) as img:
            np_img = np.array(img).flatten()
        label = int(os.path.split(fname)[-1].split('.')[0].split('_')[3])-1

        X.append(np_img)
        tempy = np.zeros(15)
        tempy[label] = 1
        Y.append(tempy)
    X, Y = np.array(X), np.array(Y)
    return(X, Y)

In [88]:
# create test, train split
X, Y = load()
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, train_size=0.8, random_state=2021)
Ytrain_classes = []
for i in range(Ytrain.shape[1]):
    Ytrain_0 = classify(Ytrain, i)
    Ytrain_classes.append(Ytrain_0)
    print(f"{Ytrain_0.count(1)} datapoints have classification {i}")

794 datapoints have classification 0
807 datapoints have classification 1
790 datapoints have classification 2
821 datapoints have classification 3
770 datapoints have classification 4
811 datapoints have classification 5
784 datapoints have classification 6
824 datapoints have classification 7
799 datapoints have classification 8
790 datapoints have classification 9
788 datapoints have classification 10
802 datapoints have classification 11
815 datapoints have classification 12
796 datapoints have classification 13
809 datapoints have classification 14


In [89]:
"""
Here we test the performace of our model with different amounts of training data 
TODO: METHOD OF CALCULATING ACCURACY MAY BE INCORRECT
"""
trainSizes = [x / 10 for x in range(1, 10, 1)] # Create a list of [0.1, 0.2, 0.3, ..., 0.9]

for trainRatio in trainSizes:
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, train_size=trainRatio, random_state=2021)
    Ytrain_classes = []
    for i in range(Ytrain.shape[1]):
        Ytrain_0 = classify(Ytrain, i)
        Ytrain_classes.append(Ytrain_0)

    model = ShallowTree()
    model.fit(Xtrain, Ytrain_classes[0])
    WL = WeakLearner(model, 0)
    WL.miss_classify(Xtrain, Ytrain_classes[0])
    WL.calc_error_rate(np.array([1 for d in range(Xtrain.shape[0])]))
    print("Percent correct with {}% training, {}% testing: {}%".format(trainRatio * 100, 100 - (trainRatio * 100), (WL.error_rate / Xtest.shape[0]) * 100))


Percent correct with 10.0% training, 90.0% testing: 0.5111111111111112%
Percent correct with 20.0% training, 80.0% testing: 1.2583333333333333%
Percent correct with 30.0% training, 70.0% testing: 2.276190476190476%
Percent correct with 40.0% training, 60.0% testing: 3.6333333333333337%
Percent correct with 50.0% training, 50.0% testing: 5.493333333333333%
Percent correct with 60.0% training, 40.0% testing: 8.566666666666666%
Percent correct with 70.0% training, 30.0% testing: 13.355555555555556%
Percent correct with 80.0% training, 20.0% testing: 22.633333333333333%
Percent correct with 90.0% training, 10.0% testing: 52.666666666666664%


In [93]:
""" 
Now we vary the size of the trees
"""
# I had to do this because of import issues
# TODO: Fix this so you don't have to redefine the function here
def ShallowTree(d = 2): 
    return DecisionTreeClassifier(max_depth=d)

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, train_size=0.8, random_state=2021)
Ytrain_classes = []
for i in range(Ytrain.shape[1]):
    Ytrain_0 = classify(Ytrain, i)
    Ytrain_classes.append(Ytrain_0)

for treeSize in range(2, 10):
    Ytrain_classes = []
    for i in range(Ytrain.shape[1]):
        Ytrain_0 = classify(Ytrain, i)
        Ytrain_classes.append(Ytrain_0)

    model = ShallowTree(treeSize)
    model.fit(Xtrain, Ytrain_classes[0])
    WL = WeakLearner(model, 0)
    WL.miss_classify(Xtrain, Ytrain_classes[0])
    WL.calc_error_rate(np.array([1 for d in range(Xtrain.shape[0])]))
    print("Percent correct with tree size {}: {}%".format(treeSize, (WL.error_rate / Xtest.shape[0]) * 100))

Percent correct with tree size 2: 22.633333333333333%
Percent correct with tree size 3: 20.200000000000003%
Percent correct with tree size 4: 16.066666666666666%
Percent correct with tree size 5: 13.433333333333334%
Percent correct with tree size 6: 10.233333333333333%
Percent correct with tree size 7: 7.433333333333334%
Percent correct with tree size 8: 6.033333333333333%
Percent correct with tree size 9: 5.066666666666666%
